In [ ]:
from imports import *
from FeatureExtraction import *
from VSM import *

## Preprocessor

In [ ]:
_PreProcessor = PreProcessor()
UC_documents, code_documents, UCTokens, CodeTokens = _PreProcessor.setup(
            "./UC", "./CC"
        )
UCTokens.update(CodeTokens)

# print(_PreProcessor.CC_to_index)

## Adjuting the csv file

In [ ]:
DataSet = pd.read_csv("eTour.csv", names=['UC', 'CC'])
temp = set()
for row in DataSet.itertuples(index=False, name=None):
    temp.add((row[0].lower(), row[1].lower()))

DataSet['Labels'] = np.ones(len(DataSet), np.uint16)

for filename_UC in os.listdir("./UC"):
    for filename_CC in os.listdir("./CC"):
        if (filename_UC.lower(), filename_CC.lower()) not in temp:
            DataSet.loc[len(DataSet)] = [filename_UC, filename_CC, 0]

DataSet.to_csv('eTourModified.csv', index = False)               

## Reading the dataset

In [ ]:
DataSet = pd.read_csv("eTourModified.csv")
# print(DataSet)
##### incorrect data / duplicated data / NA 
# DataSet.dropna(inplace=True)
# print(DataSet.duplicated().to_string())

###### TO DO: Add the rest of the negative class

for row in DataSet.index:
    UC = DataSet.loc[row, 'UC'].lower()
    CC = DataSet.loc[row, 'CC'].lower()
    if UC in _PreProcessor.UC_to_index:
        DataSet.loc[row, 'UC'] = _PreProcessor.UC_to_index[UC]
    if CC in _PreProcessor.CC_to_index:
        DataSet.loc[row, 'CC'] = _PreProcessor.CC_to_index[CC]

print(DataSet.to_string())

## Collecting the 131 features

In [ ]:
featureExtraction = FeatureExtraction(UCTokens)

###### used for all ######
tfidf_matrix_uc, tfidf_matrix_code,idf_uc_dict,idf_code_dict,feature_names_uc,feature_names_code ,df_uc_dict,df_code_dict= featureExtraction.TFIDFVectorizer(UC_documents, code_documents)
UC_count_matrix, code_count_matrix,tf_uc_dict,tf_code_dict = featureExtraction.CountVectorizerModel(UC_documents, code_documents, 'train')
idf_uc_q,idf_code_q= featureExtraction.IDFPreProcessing(UC_documents,idf_code_dict,code_documents,idf_uc_dict)
ictf_uc_q,ictf_code_q=featureExtraction.ICTFPreProcessing(UC_documents,tf_code_dict,code_documents,tf_uc_dict)
entropy_uc,entropy_code=featureExtraction.EntropyPreProcessing(UC_documents,code_documents,df_uc_dict,df_code_dict)
variance_uc,variance_code= featureExtraction.VarPreProcessing(UC_documents,code_documents,idf_uc_dict,idf_code_dict)
PMI_uc,PMI_code=featureExtraction.PMIPreProcessing(code_documents,UC_documents)
SCQ_uc,SCQ_code = featureExtraction.SCQPreProcessing(UC_documents,code_documents,tf_uc_dict,tf_code_dict,idf_uc_dict,idf_code_dict)

# the values of the count matrices are normalized

#-------------------------14 IR based features--------------------------#

# 1) Vector space model
cosine_similarities_feature = featureExtraction.VectorSpaceModel(tfidf_matrix_uc, tfidf_matrix_code)
print("cosine_similarities_feature", cosine_similarities_feature.shape)

# Our model will classify based on the ranks of the data rather than their actual values.
cosine_similarity_UC = rankdata(-cosine_similarities_feature, method='dense', axis=1)
print("cosine_similarity_UC", cosine_similarity_UC.shape)
cosine_similarity_CC = rankdata(-cosine_similarities_feature, method='dense', axis=0)
print("cosine_similarity_CC", cosine_similarity_CC.shape)

# 2) Latent semantic analysis
LSA_similraities_feature = featureExtraction.LSA(tfidf_matrix_uc, tfidf_matrix_code)
print('LSA similarity', LSA_similraities_feature.shape)

LSA_similraities_UC = rankdata(-LSA_similraities_feature, method='dense', axis=1)
print("LSA_similraities_UC", LSA_similraities_UC.shape)
LSA_similraities_CC = rankdata(-LSA_similraities_feature, method='dense', axis=0)
print("LSA_similraities_CC", LSA_similraities_CC.shape)

# 3) Latent Dirichlet Allocation
DocumentTopicDisUC_dense, DocumentTopicDisCode_dense, cosine_similarities_LDA= featureExtraction.LDA(UC_documents,code_documents,UCTokens)
print('LDA similarity', cosine_similarities_LDA.shape)

LDA_similraities_UC = rankdata(-cosine_similarities_LDA, method='dense', axis=1)
print("LDA_similraities_UC", LDA_similraities_UC.shape)
LDA_similraities_CC = rankdata(-cosine_similarities_LDA, method='dense', axis=0)
print("LDA_similraities_CC", LDA_similraities_CC.shape)

# 4) Jensen-Shannon(JS)
JS_features = featureExtraction.JensenShannon(UC_count_matrix, code_count_matrix)
print('JS', JS_features.shape)

JS_UC = rankdata(-JS_features, method='dense', axis=1)
print('JS_UC', JS_UC.shape)
JS_CC = rankdata(-JS_features, method='dense', axis=0)
print('JS_CC', JS_CC.shape)

# 5)  Okapi BM25
# UC means query is UC and document is code
BM25_UC = featureExtraction.BM25(UC_documents,code_documents,idf_code_dict,code_count_matrix)
BM25_CC = featureExtraction.BM25(code_documents,UC_documents,idf_uc_dict,UC_count_matrix)

BM25_UC = rankdata(-BM25_UC, method='dense', axis=0)
print("BM25_UC", BM25_UC.shape)
BM25_CC = rankdata(-BM25_CC, method='dense', axis=0)
print("BM25_CC", BM25_CC.shape)

# 6) Language Model with Dirichlet
# UC means UC is query and code is doc
JM_UC = featureExtraction.SmoothingMethods(UC_documents,code_documents,code_count_matrix,tf_code_dict,JM_or_DP=True)
JM_CC = featureExtraction.SmoothingMethods(code_documents,UC_documents,UC_count_matrix,tf_uc_dict,True)

DP_UC = featureExtraction.SmoothingMethods(UC_documents,code_documents,code_count_matrix,tf_code_dict,JM_or_DP=False)
DP_CC = featureExtraction.SmoothingMethods(code_documents,UC_documents,UC_count_matrix,tf_uc_dict,False)

JM_UC = rankdata(-JM_UC, method='dense', axis=0)
print("JM_UC", JM_UC.shape)
JM_CC = rankdata(-JM_CC, method='dense', axis=0)
print("JM_CC", JM_CC.shape)
DP_UC = rankdata(-DP_UC, method='dense', axis=0)
print("DP_UC", DP_UC.shape)
DP_CC = rankdata(-DP_CC, method='dense', axis=0)
print("DP_CC", DP_CC.shape)

# ------------------------pre-retrieval (21 metrics)--------------------------#
# 1) IDF Features
avg_idf_uc= featureExtraction.AvgIDF(idf_uc_q)
avg_idf_code= featureExtraction.AvgIDF(idf_code_q)
print('avg_idf_uc_shape', avg_idf_uc.shape) 
print('avg_idf_code_shape', avg_idf_code.shape) 
max_idf_uc= featureExtraction.MaxIDF(idf_uc_q)
max_idf_code= featureExtraction.MaxIDF(idf_code_q)
print('max_idf_uc_shape', max_idf_uc.shape) 
print('max_idf_code_shape', max_idf_code.shape) 

dev_idf_uc= featureExtraction.DevIDF(idf_uc_q)
dev_idf_code= featureExtraction.DevIDF(idf_code_q)

print('dev_idf_uc_shape', dev_idf_uc.shape) 
print('dev_idf_code_shape', dev_idf_code.shape) 

# 2) ICTF Features
avg_ictf_uc= featureExtraction.AvgICTF(ictf_uc_q)
avg_ictf_code= featureExtraction.AvgICTF(ictf_code_q)

print('avg_ictf_uc_shape', avg_ictf_uc.shape)
print('avg_ictf_code_shape', avg_ictf_code.shape)

max_ictf_uc= featureExtraction.MaxICTF(ictf_uc_q)
max_ictf_code= featureExtraction.MaxICTF(ictf_code_q)

print('max_ictf_uc_shape', max_ictf_uc.shape)
print('max_ictf_code_shape', max_ictf_code.shape)

dev_ictf_uc= featureExtraction.DevICTF(ictf_uc_q)
dev_ictf_code= featureExtraction.DevICTF(ictf_code_q)

print('dev_ictf_uc_shape', dev_ictf_uc.shape)
print('dev_ictf_code_shape', dev_ictf_code.shape)
# 3) Entropy Features

avg_entropy_uc= featureExtraction.AvgEntropy(entropy_uc)
avg_entropy_code= featureExtraction.AvgEntropy(entropy_code)

print('avg_entropy_uc_shape', avg_entropy_uc.shape)
print('avg_entropy_code_shape', avg_entropy_code.shape)

max_entropy_uc= featureExtraction.MaxEntropy(entropy_uc)
max_entropy_code= featureExtraction.MaxEntropy(entropy_code)

print('max_entropy_uc_shape', max_entropy_uc.shape)
print('max_entropy_code_shape', max_entropy_code.shape)

med_entropy_uc= featureExtraction.MedEntropy(entropy_uc)
med_entropy_code= featureExtraction.MedEntropy(entropy_code)

print('med_entropy_uc_shape', med_entropy_uc.shape)
print('med_entropy_code_shape', med_entropy_code.shape)

dev_entropy_uc= featureExtraction.DevEntropy(entropy_uc)
dev_entropy_code= featureExtraction.DevEntropy(entropy_code)

print('dev_entropy_uc_shape', dev_entropy_uc.shape)
print('dev_entropy_code_shape', dev_entropy_code.shape)
      

# 4) Variance Features

avg_variance_uc= featureExtraction.AvgVariance(variance_uc) 
avg_variance_code= featureExtraction.AvgVariance(variance_code)

print('avg_variance_uc_shape', avg_variance_uc.shape)
print('avg_variance_code_shape', avg_variance_code.shape)

max_variance_uc= featureExtraction.MaxVariance(variance_uc)
max_variance_code= featureExtraction.MaxVariance(variance_code)

print('max_variance_uc_shape', max_variance_uc.shape)
print('max_variance_code_shape', max_variance_code.shape)

sum_variance_uc= featureExtraction.SumVariance(variance_uc)
sum_variance_code= featureExtraction.SumVariance(variance_code)

print('sum_variance_uc_shape', sum_variance_uc.shape)
print('sum_variance_code_shape', sum_variance_code.shape)
# 5) SCQ Features
avg_scq_uc= featureExtraction.AvgSCQ(SCQ_uc)
avg_scq_code= featureExtraction.AvgSCQ(SCQ_code)

print('avg_scq_uc_shape', avg_scq_uc.shape)
print('avg_scq_code_shape', avg_scq_code.shape)

max_scq_uc= featureExtraction.MaxSCQ(SCQ_uc)
max_scq_code= featureExtraction.MaxSCQ(SCQ_code)

print('max_scq_uc_shape', max_scq_uc.shape)
print('max_scq_code_shape', max_scq_code.shape)

sum_sqc_uc= featureExtraction.SumSCQ(SCQ_uc)
sum_sqc_code= featureExtraction.SumSCQ(SCQ_code)

print('sum_sqc_uc_shape', sum_sqc_uc.shape)
print('sum_sqc_code_shape', sum_sqc_code.shape)

# 6) PMI Features

avg_pmi_uc= featureExtraction.AvgPMI(PMI_uc)
avg_pmi_code= featureExtraction.AvgPMI(PMI_code)

print('avg_pmi_uc_shape', avg_pmi_uc.shape)
print('avg_pmi_code_shape', avg_pmi_code.shape)

max_pmi_uc= featureExtraction.MaxPMI(PMI_uc)
max_pmi_code= featureExtraction.MaxPMI(PMI_code)

print('max_pmi_uc_shape', max_pmi_uc.shape)
print('max_pmi_code_shape', max_pmi_code.shape)
# 7) QS Features

qs_uc= featureExtraction.QS(UC_documents,code_documents)
print('qs_uc_shape', qs_uc.shape)

qs_code = featureExtraction.QS(code_documents,UC_documents)
print('qs_code_shape', qs_code.shape)



#------------------------post-retrieval (7 metrics)--------------------------#
# 1.1) Subquery overlap using jensenShannon
code_queries_score_JensenShannon = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.JensenShannon,"JS", code_count_matrix, UC_count_matrix)
UC_queries_score_JensenShannon = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.JensenShannon,"JS",code_count_matrix, UC_count_matrix)

print('code_queries_score_JensenShannon', code_queries_score_JensenShannon.shape)
print('UC_queries_score_JensenShannon', UC_queries_score_JensenShannon.shape)

# 1.2) Subquery overlap using VSM
code_queries_score_VSM = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.VectorSpaceModel,"VSM", UC_count_matrix, code_count_matrix)
UC_queries_score_VSM = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.VectorSpaceModel,"VSM", code_count_matrix, UC_count_matrix)

print('code_queries_score_VSM', code_queries_score_VSM.shape)
print('UC_queries_score_VSM', UC_queries_score_VSM.shape)

# 1.3) Subquery overlap using BM25
code_queries_score_BM25 = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.BM25,"BM", UC_documents,idf_uc_dict,UC_count_matrix)
UC_queries_score_BM25 = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.BM25,"BM", code_documents,idf_code_dict, code_count_matrix)

print('code_queries_score_BM25', code_queries_score_BM25.shape)
print('UC_queries_score_BM25', UC_queries_score_BM25.shape)

# 1.4) Subquery overlap using JM smoothing
code_queries_score_JM = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.SmoothingMethods,"JM", UC_documents,UC_count_matrix,tf_uc_dict,JM_or_DP=True)
UC_queries_score_JM = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.SmoothingMethods,"JM", code_documents,code_count_matrix,tf_code_dict,JM_or_DP=True)

print('code_queries_score_JM', code_queries_score_JM.shape)
print('UC_queries_score_JM', UC_queries_score_JM.shape)

# 1.5) Subquery overlap using DP smoothing
code_queries_score_DP = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.SmoothingMethods,"DP", UC_count_matrix, code_count_matrix,tf_code_dict,JM_or_DP=False)
UC_queries_score_DP = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.SmoothingMethods,"DP", code_count_matrix, UC_count_matrix,tf_uc_dict,JM_or_DP=False)

print('code_queries_score_DP', code_queries_score_DP.shape)
print('UC_queries_score_DP', UC_queries_score_DP.shape)

# 6.1) WIG using JensenShannon
UC_WIG_score_JensenShannon = featureExtraction.WeightedInformationGain(UC_documents,code_documents,JS_UC.T,tf_code_dict,np.sum(tf_code_dict.values()))
code_WIG_score_JensenShannon = featureExtraction.WeightedInformationGain(code_documents,UC_documents,JS_CC,tf_uc_dict,np.sum(tf_uc_dict.values()))

# 6.2) WIG Score using VSM 
UC_WIG_score_VSM = featureExtraction.WeightedInformationGain(UC_documents,code_documents,cosine_similarity_UC.T,tf_code_dict,np.sum(tf_code_dict.values()))
code_WIG_score_VSM = featureExtraction.WeightedInformationGain(code_documents,UC_documents,cosine_similarity_CC,tf_uc_dict,np.sum(tf_uc_dict.values()))

# 6.3) WIG Score using BM25
UC_WIG_score_BM25 = featureExtraction.WeightedInformationGain(UC_documents,code_documents,BM25_UC,tf_code_dict,np.sum(tf_code_dict.values()))
code_WIG_score_BM25 = featureExtraction.WeightedInformationGain(code_documents,UC_documents,BM25_CC,tf_uc_dict,np.sum(tf_uc_dict.values()))

# 7.1) NQC using JensenShannon
UC_NQC_JensenShannon = featureExtraction.NormalizedQueryCommitment(JS_UC.T)
code_NQC_JensenShannon = featureExtraction.NormalizedQueryCommitment(JS_CC)

print("UC_NQC_JensenShannon",UC_NQC_JensenShannon.shape)
print("code_NQC_JensenShannon",code_NQC_JensenShannon.shape)

# 7.2) NQC using VSM
UC_NQC_VSM = featureExtraction.NormalizedQueryCommitment(cosine_similarity_UC.T)
code_NQC_VSM = featureExtraction.NormalizedQueryCommitment(cosine_similarity_CC)

print("UC_NQC_VSM",UC_NQC_VSM.shape)
print("code_NQC_VSM",code_NQC_VSM.shape)

# 7.3) NQC using BM25
UC_NQC_BM25 = featureExtraction.NormalizedQueryCommitment(BM25_UC)
code_NQC_BM25 = featureExtraction.NormalizedQueryCommitment(BM25_CC)

print("UC_NQC_BM25",UC_NQC_BM25.shape)
print("code_NQC_BM25",code_NQC_BM25.shape)

#------------------------Document Statistics Features--------------------------#
num_terms_code, num_terms_UC, num_unique_terms_code, num_unique_terms_UC, num_overlapping_terms = featureExtraction.DocumentStatistics(UC_documents, code_documents)



In [ ]:
###### Remove hardcoded sizing with the size captured from the data set.
avg_idf_uc_reshaped = np.tile(avg_idf_uc, (116,1))
avg_idf_code_reshaped = np.tile(avg_idf_code, (58,1))

max_idf_uc_reshaped = np.tile(max_idf_uc, (116,1))
max_idf_code_reshaped = np.tile(max_idf_code, (58,1))

dev_idf_uc_reshaped = np.tile(dev_idf_uc, (116,1))
dev_idf_code_reshaped = np.tile(dev_idf_code, (58,1))

avg_ictf_uc_reshaped = np.tile(avg_ictf_uc, (116, 1))
avg_ictf_code_reshaped = np.tile(avg_ictf_code, (58, 1))

max_ictf_uc_reshaped = np.tile(max_ictf_uc, (116, 1))
max_ictf_code_reshaped = np.tile(max_ictf_code, (58, 1))

dev_ictf_uc_reshaped = np.tile(dev_ictf_uc, (116, 1))
dev_ictf_code_reshaped = np.tile(dev_ictf_code, (58, 1))

avg_entropy_uc_reshaped = np.tile(avg_entropy_uc, (116, 1))
avg_entropy_code_reshaped = np.tile(avg_entropy_code, (58, 1))

max_entropy_uc_reshaped = np.tile(max_entropy_uc, (116, 1)) # ------> gets removed, 2nd round
max_entropy_code_reshaped = np.tile(max_entropy_code, (58, 1)) # ------> gets removed, 2nd round

med_entropy_uc_reshaped = np.tile(med_entropy_uc, (116, 1))
med_entropy_code_reshaped = np.tile(med_entropy_code, (58, 1))

dev_entropy_uc_reshaped = np.tile(dev_entropy_uc, (116, 1))
dev_entropy_code_reshaped = np.tile(dev_entropy_code, (58, 1)) # -------> gets removed

avg_variance_uc_reshaped = np.tile(avg_variance_uc, (116, 1))
avg_variance_code_reshaped = np.tile(avg_variance_code, (58, 1))

max_variance_uc_reshaped = np.tile(max_variance_uc, (116, 1))
max_variance_code_reshaped = np.tile(max_variance_code, (58, 1))

sum_variance_uc_reshaped = np.tile(sum_variance_uc, (116, 1)) # --------> gets removed
sum_variance_code_reshaped = np.tile(sum_variance_code, (58, 1)) # -------> gets removed

avg_scq_uc_reshaped = np.tile(avg_scq_uc, (116, 1))
avg_scq_code_reshaped = np.tile(avg_scq_code, (58, 1)) # -------> gets removed

max_scq_uc_reshaped = np.tile(max_scq_uc, (116, 1))
max_scq_code_reshaped = np.tile(max_scq_code, (58, 1))

sum_sqc_uc_reshaped = np.tile(sum_sqc_uc, (116, 1)) # --------> gets removed, 2nd round
sum_sqc_code_reshaped = np.tile(sum_sqc_code, (58, 1)) # --------> gets removed, 2nd round

avg_pmi_uc_reshaped = np.tile(avg_pmi_uc, (116, 1)) # --------> gets removed 
avg_pmi_code_reshaped = np.tile(avg_pmi_code, (58, 1)) # --------> gets removed 

max_pmi_uc_reshaped = np.tile(max_pmi_uc, (116, 1))
max_pmi_code_reshaped = np.tile(max_pmi_code, (58, 1))

qs_uc_reshaped = np.tile(qs_uc, (116, 1)) # --------> gets removed, 2nd round
qs_code_reshaped = np.tile(qs_code, (58, 1)) # --------> gets removed, 2nd round

feature_matrix = np.stack((cosine_similarity_UC, cosine_similarity_CC, LSA_similraities_UC, LSA_similraities_CC,
                           JS_UC, JS_CC,LDA_similraities_UC,LDA_similraities_CC, avg_idf_uc_reshaped.T, avg_idf_code_reshaped, max_idf_uc_reshaped.T,
                           max_idf_code_reshaped, dev_idf_uc_reshaped.T, dev_idf_code_reshaped, avg_ictf_uc_reshaped.T,
                           avg_ictf_code_reshaped, max_ictf_uc_reshaped.T, max_ictf_code_reshaped, dev_ictf_uc_reshaped.T,
                           dev_ictf_code_reshaped, avg_entropy_uc_reshaped.T, avg_entropy_code_reshaped, max_entropy_uc_reshaped.T,
                           max_entropy_code_reshaped, med_entropy_uc_reshaped.T, med_entropy_code_reshaped, dev_entropy_uc_reshaped.T, 
                           dev_entropy_code_reshaped, avg_variance_uc_reshaped.T, avg_variance_code_reshaped, max_variance_uc_reshaped.T,
                           max_variance_code_reshaped, sum_variance_uc_reshaped.T, sum_variance_code_reshaped, avg_scq_uc_reshaped.T,
                           avg_scq_code_reshaped, max_scq_uc_reshaped.T, max_scq_code_reshaped, sum_sqc_uc_reshaped.T, sum_sqc_code_reshaped,
                           avg_pmi_uc_reshaped.T, avg_pmi_code_reshaped, max_pmi_uc_reshaped.T, max_pmi_code_reshaped, qs_uc_reshaped.T, 
                           qs_code_reshaped), axis=2)

print(feature_matrix.shape)


In [ ]:
# num_components = min(tfidf_matrix_uc.shape[0], tfidf_matrix_code.shape[1])
# num_components = min(num_components, 100)
# LSA_model = TruncatedSVD(n_components=num_components)

# # if train_or_test == "train":
# LSA_data_useCases = LSA_model.fit_transform(tfidf_matrix_uc)
# LSA_data_codes = LSA_model.fit_transform(tfidf_matrix_code) 
# # else:
# #     LSA_data_useCases = LSA_model.transform(tfidf_matrix_uc)
# #     LSA_data_codes = LSA_model.transform(tfidf_matrix_code) 

# LSA_similraity_matrix = cosine_similarity(LSA_data_useCases, LSA_data_codes)
# print(LSA_similraity_matrix)

In [ ]:
# test = FeatureExtraction()
# test.LSA(tfidf_matrix_uc, tfidf_matrix_code, "train")

In [ ]:
#usecases (100*usecases_nu)
#code (100*codes_nu)
#LSA_similraity_matrix (usecases_nu*codes_nu)
#d1,d2,d3,d4
#u1,u2,u3,u4
#u1*d1,u1*d2,u1*d3 ...
#d1*u1,d1*u2,d1*u3 ...
#each row coresspond between the use cases and each document
# [[d1*c1 d1*c2],[d2*c1 d2*c2]]

# LSA_data_codes=np.array([[1,2,3,4,5] , [6,7,8,9,10]])
# LSA_data_useCases=np.array([[11,12,13,14,15] , [16,17,18,19,21],[2,5,9,11,42]])

# LSA_data_codes_norm=np.repeat(np.linalg.norm(LSA_data_codes,axis=1),LSA_data_useCases.shape[0])
# LSA_data_useCases_norm=np.repeat(np.linalg.norm(LSA_data_useCases,axis=1),LSA_data_codes.shape[0])

# LSA_data_codes_norm=LSA_data_codes_norm.reshape(LSA_data_codes.shape[0],LSA_data_useCases.shape[0]).T 
# LSA_data_useCases_norm=LSA_data_useCases_norm.reshape(LSA_data_useCases.shape[0],LSA_data_codes.shape[0])

# print(LSA_data_codes_norm)
# print(LSA_data_useCases_norm)

# LSA_similraity_matrix=cosine_similarity(LSA_data_useCases, LSA_data_codes)

# print(LSA_similraity_matrix)

## Jensen-Shannon

In [ ]:
test = FeatureExtraction()
test.JensenShannon(UCTokens, UC_documents, code_documents)

In [ ]:
def getOverlap(query_term_list: list, total_query_list: list) -> int:
    query_term_list = pd.Series(query_term_list)
    query_term_list = query_term_list.nlargest(10)
    query_term_set = set(query_term_list.index.values)

    total_query_list = pd.Series(total_query_list)
    total_query_list = total_query_list.nlargest(10)
    total_query_set = set(total_query_list.index.values)

    return(len(query_term_set & total_query_set))



#running the query as a whole
feature_extraction=FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

### Run the original query q, and obtain the result list R 
total_score_whole_query_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, code_count_matrix) 

### Run each individual query term qt in the original query as a separate query and obtain the result list Rt.
overall_queries_score = list()
for code_idx, code in enumerate(code_documents):
    code_words = code.split(" ")
    query__term_overlap_list = list()
    for word in code_words:
        if word == '':
            continue
        # word_count_matrix = np.zeros((code_count_matrix.shape[0], code_count_matrix.shape[1]))
        word_index = feature_extraction.code_vocab_index[word]
        # word_count_matrix[:, word_index] = code_count_matrix[:, word_index]
        # print(feature_extraction.count_vectorizer.vocabulary_["if"])
        # print(code_count_matrix[:, 752])
        # print(code_count_matrix[:, word_index])
        term_vector=np.zeros((1, UC_count_matrix.shape[1]))
        term_vector[:,word_index]=code_count_matrix[code_idx, word_index]
        query_term_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, term_vector)
        # print(query_term_jensen_shannon.shape)
        query_term_len = query_term_jensen_shannon.shape[0] * query_term_jensen_shannon.shape[1]
        query__term_overlap_list.append(getOverlap(query_term_jensen_shannon.reshape(query_term_len), total_score_whole_query_jensen_shannon[:, code_idx]))

    overall_queries_score.append(stdev(query__term_overlap_list))
# print(overall_queries_score)
        # print(total_score_query_term_jensen_shannon)

        
    # break




# print(total_score_whole_query_jensen_shannon[:,7])
# print(total_score_whole_query_jensen_shannon[:,6])
# print(code[7])
# for i, temp in enumerate(code[7]):
#     if temp != 0:
#         print(i)
# # 3 features to be called
#1st case => if the query is the code
# total_score_whole_query_jensen_shannon=feature_extraction.JensenShannon(UC_documents,,"train") #58*116

In [ ]:
# def getOverlap(query_term_list: np.ndarray, total_query_set: set) -> int:
#     query_term_set = set(np.argpartition(query_term_list, -10)[-10:])
#     return len(query_term_set & total_query_set)

# def SubqueryOverlapTerms(query_term: str, code_idx: int, UC_count_matrix: np.ndarray, code_count_matrix: np.ndarray, feature_extraction: object) -> np.ndarray:
    
#     word_index = feature_extraction.code_vocab_index[query_term]
#     term_vector = np.zeros((1, UC_count_matrix.shape[1])) #1*
#     term_vector[:,word_index]=code_count_matrix[code_idx, word_index]

#     query_term_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, term_vector)
#     query_term_len = query_term_jensen_shannon.shape[0] * query_term_jensen_shannon.shape[1]

#     query_term_jensen_shannon = query_term_jensen_shannon.reshape(query_term_len)
#     return query_term_jensen_shannon

# def SubqueryOverlapCode(code_idx, code, UC_count_matrix: np.ndarray, code_count_matrix: np.ndarray, feature_extraction: object, total_query_list: np.ndarray):

#     code_idx = int(code_idx)
#     code_words = code.split(" ")
#     code_words = np.ascontiguousarray(code_words[0:-1])
#     vSubqueryOverlap = np.vectorize(lambda query_term: SubqueryOverlapTerms(query_term, code_idx, UC_count_matrix, code_count_matrix, feature_extraction), otypes=[np.ndarray])
    
#     query_term_jensen_shannon = vSubqueryOverlap(code_words)
#     total_query_set = set(np.argpartition(total_query_list[:, code_idx], -10)[-10:])
   
#     vGetOverlap = np.vectorize(lambda query_term_jensen_shannon: getOverlap(query_term_jensen_shannon, total_query_set), otypes=[np.int16])
#     query_term_overlap = vGetOverlap(query_term_jensen_shannon)
    
#     overall_queries_score = np.std(query_term_overlap)
#     return overall_queries_score

In [ ]:
#feature_extraction.JensenShannon
# def SubqueryOverlap(feature_extraction:object, query:list, feature_extraction_method:Callable, document_count_matrix:np.ndarray, query_count_matrix:np.ndarray) -> np.ndarray:
    
#     ### Run the original query q, and obtain the result list R 
#     total_score_query_feature_extraction = feature_extraction_method(document_count_matrix, query_count_matrix)

#     query_tuples =  np.ascontiguousarray(list(enumerate(query)))
    
#     ### Run each individual query term qt in the original query as a separate query and obtain the result list Rt.
#     vSubqueryOverlapCode = np.vectorize(lambda code_idx, code_doc: SubqueryOverlapCode(code_idx, code_doc, document_count_matrix, query_count_matrix, feature_extraction, total_score_query_feature_extraction), otypes=[np.float16])

#     overall_queries_score = vSubqueryOverlapCode(query_tuples[:,0], query_tuples[:,1])
#     return overall_queries_score

## Subquery overlap test

In [ ]:
#client = Client(processes=False, threads_per_worker=4,n_workers=2, memory_limit='2GB')

feature_extraction = FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

 
code_queries_score_JensenShannon = feature_extraction.SubqueryOverlap(feature_extraction, code_documents, feature_extraction.JensenShannon, UC_count_matrix, code_count_matrix)
UC_queries_score_JensenShannon = feature_extraction.SubqueryOverlap(feature_extraction, UC_documents, feature_extraction.JensenShannon, code_count_matrix, UC_count_matrix)

# tfidf_matrix_uc,tfidf_matrix_code=feature_extraction.TFIDFVectorizer(UC_documents,code_documents)
# # code_queries_score_vector_space = feature_extraction.SubqueryOverlap(feature_extraction, code_documents,cosine_similarity, tfidf_matrix_uc, #tfidf_matrix_code)
# uc_queries_score_vector_space = feature_extraction.SubqueryOverlap(feature_extraction, UC_documents,cosine_similarity, tfidf_matrix_code,tfidf_matrix_uc)

# # print(code_queries_score_vector_space.shape)
# print(uc_queries_score_vector_space.shape)
# print(uc_queries_score_vector_space)


In [ ]:
feature_extraction=FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

### Run the original query q, and obtain the result list R 
total_score_whole_query_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, code_count_matrix) 
query_term_set = np.argsort(total_score_whole_query_jensen_shannon,axis=0)[-50:,:]
new_ranked_documents=[] #50 documents

print(query_term_set.shape)
# print(total_score_whole_query_jensen_shannon[0])

In [ ]:
def get_overlap(query_term_list: np.ndarray, total_query_list: np.ndarray) -> int:
    query_term_set = set(np.argpartition(-query_term_list, 10)[:10])
    total_query_set = set(np.argpartition(-total_query_list, 10)[:10])

    return len(query_term_set & total_query_set)

temp1 = np.array([90, 100, 200, 30, 400, 500, 600, 700, 800, 900, 100])
temp2 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

np.argsort(-temp1)[:10]

# temp1 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# temp2 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# indices_temp1 = np.argpartition(-temp1, 11)[:10]
# sorted_indices_temp1 = indices_temp1[np.argsort(temp1[indices_temp1])]

# print(sorted_indices_temp1)


### Document Statistics Features

In [ ]:
num_terms_code =[]
for document in code_documents:
    num_terms_code.append(len(document.split()))

num_terms_code=np.array(num_terms_code)
print(num_terms_code)

num_terms_UC =[]
for document in UC_documents:
    num_terms_UC.append(len(document.split()))

num_terms_UC=np.array(num_terms_UC)
print(num_terms_UC)

num_unique_terms_code =[]
for document in code_documents:
    num_unique_terms_code.append(len(set(document.split())))

num_unique_terms_code=np.array(num_unique_terms_code)
print(num_unique_terms_code)

num_unique_terms_UC =[]
for document in UC_documents:
    num_unique_terms_UC.append(len(set(document.split())))

num_unique_terms_UC=np.array(num_unique_terms_UC)
print(num_unique_terms_UC)

# each column represent D1 intersect with all usecases, then column2= D2 intersect with all usescases....
#if want to access the intesection between a specific usecase and all coressponding documents you will access the row index coressponding to this usecase 
num_overlapping_terms_code =np.zeros((len(UC_documents),len(code_documents)))
for uc_index,uc in enumerate(UC_documents):
    for code_index,code in enumerate(code_documents):
        num_overlapping_terms_code[uc_index][code_index] = 100*(len(set(uc.split()).intersection(set(code.split()))))/(num_unique_terms_UC[uc_index]+num_unique_terms_code[code_index])

print(num_overlapping_terms_code)




## Feature Selection

In [ ]:
# matrix shape = links * features 
# ----features------
#|
#links
#|
feature_matrix_reshaped = feature_matrix.reshape(feature_matrix.shape[0]*feature_matrix.shape[1], -1)
print(feature_matrix_reshaped.shape)
correlation_features=np.corrcoef(feature_matrix_reshaped,rowvar=False)
print(correlation_features.shape)
features_excluded=set()

for i in range(correlation_features.shape[1]):
    for j in range(i+1,correlation_features.shape[0]):
        if (correlation_features[j][i] >= 0.9):
            print(j,i)
            features_excluded.add(j)

features_links_selected=np.delete(feature_matrix_reshaped, list(features_excluded), axis=1) 
print(features_excluded)
print(features_links_selected.shape)
features_links_selected_reshaped = features_links_selected.reshape(feature_matrix.shape[0], feature_matrix.shape[1], -1)
print(features_links_selected_reshaped.shape)
# print(correlation_features)
# for i in correlation_features:
#     print (i)


### Mapping Features to Labels

In [ ]:
Features = list()
for row in DataSet.index:
    index_code = DataSet.loc[row, 'CC']
    index_UC = DataSet.loc[row, 'UC']
    Features.append(features_links_selected_reshaped[index_UC][index_code])


## Data Imbalancing

In [ ]:
from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state=42)
Features_SMOTE, Labels_SMOTE = smote.fit_resample(Features, DataSet['Labels'].to_list() )

## Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

train_features, test_features, train_labels, test_labels = train_test_split(Features_SMOTE, Labels_SMOTE, test_size = 0.25, random_state = 42)
model_random_forest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
model_random_forest.fit(train_features, train_labels)

predictions = model_random_forest.predict(test_features)
errors = abs(predictions - test_labels)
print('Mean Absolute Error:', round(np.mean(errors), 2) )


In [ ]:
acc=(test_labels==predictions).sum()
print(acc/len(test_labels))

In [ ]:
import csv
with open('AnswerSet.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    valid_links_labels = []
    for row in reader:
        temp=row[0].split(",")
        #match = re.search(r'UC(\d+)\.txt', temp[0])
        valid_links_labels.append((temp[0],temp[1]))
        # if (valid_links_labels.get(temp[1])==None):
        # valid_links_labels[temp[1]]=[int(match.group(1))]
        # else:
        #     valid_links_labels[temp[1]].append(int(match.group(1)))

# file_names = list(valid_links_labels.keys())
# file_names.sort()
# valid_links_labels_sorted = {i: valid_links_labels[i] for i in file_names}
            
# print(valid_links_labels_sorted)
# print(len(valid_links_labels_sorted.keys()))
print(valid_links_labels)